In [5]:
import os
import sys
import tensorflow as tf
import cv2
import numpy as np
import json
import codecs
from collections import OrderedDict
import matplotlib.pyplot as plt
from PIL import Image

!pip list | grep 'tensorflow'
!pip list | grep 'opencv-python'
!pip list | grep 'numpy'
!pip list | grep 'matplotlib'

tensorflow          1.11.0                
You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
opencv-python       3.4.3.18              
You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
numpy               1.15.2                
You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
matplotlib          3.0.0                 
You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


# Neural Network Concatenate

In [2]:
"""
y = g_1(x)
z = g_2(y)
"""
with tf.Graph().as_default() as g_1:
    input = tf.placeholder(tf.float32, name="input")
    y = tf.add(input, tf.constant(1.))
    output = tf.identity(y, name="output")  # NOTE: using identity to get a known name for the output tensor.
gdef_1 = g_1.as_graph_def()


with tf.Graph().as_default() as g_2:  # NOTE: g_2 not g_1       
    input = tf.placeholder(tf.float32, name="input")
    z = tf.add(input, tf.constant(20.))
    output = tf.identity(z, name="output")
gdef_2 = g_2.as_graph_def()


combine_graph = tf.Graph()
with combine_graph.as_default():
    x = tf.placeholder("float", name="x")

    # Import gdef_1, which performs f(x).
    # "input:0" and "output:0" are the names of tensors in gdef_1.
    y, = tf.import_graph_def(gdef_1, input_map={"input:0": x}, return_elements=["output:0"])

    # Import gdef_2, which performs g(y)
    z, = tf.import_graph_def(gdef_2, input_map={"input:0": y}, return_elements=["output:0"])
    
with combine_graph.as_default():
    with tf.Session() as sess:
        opts = tf.get_default_graph().get_operations()
        for opt in opts:
            for output in opt.outputs: print(output.name)
                
        print(sess.run(z, feed_dict={x:1}))

x:0
import/input:0
import/Const:0
import/Add:0
import/output:0
import_1/input:0
import_1/Const:0
import_1/Add:0
import_1/output:0
22.0


# Operations Replacement

Include inserting, deleting a section of operations.

In [2]:
with tf.Graph().as_default() as g_3:
    """
    output = (input + 1) * 10 - 2 + 100
    """
    input = tf.placeholder(tf.float32, name="input")
    mid = tf.add(input, tf.constant(1.))
    mul = tf.multiply(mid, tf.constant(10.,), name="mul")
    y = tf.subtract(mul, tf.constant(2.), name="sub")  # <-- replaced node
    y_2 = tf.add(y, tf.constant(100.))
    output = tf.identity(y_2, name="output")  # NOTE: using identity to get a known name for the output tensor.
gdef_3 = g_3.as_graph_def()

In [3]:
replace_before_graph = tf.Graph()
with replace_before_graph.as_default():
    tf.import_graph_def(gdef_3)
    with tf.Session() as sess:
        opts = tf.get_default_graph().get_operations()
        print("----- Origin Graph -----")
        for opt in opts:
            for output in opt.outputs: print(output.name)

----- Origin Graph -----
import/input:0
import/Const:0
import/Add:0
import/Const_1:0
import/mul:0
import/Const_2:0
import/sub:0
import/Const_3:0
import/Add_1:0
import/output:0


In [4]:
combine_graph_2 = tf.Graph()
with combine_graph_2.as_default():
    """
    origin : output = (input + 1) * 10 - 2 + 100
    to : output = (input + 1) * 10 * 2 + 100
    """
    
    x = tf.placeholder("float", name="x")

    mul, = tf.import_graph_def(gdef_3, input_map={"input:0": x}, return_elements=["mul:0"])
    mul_2 = tf.multiply(mul, tf.constant(2.))  # <-- new node
    output, = tf.import_graph_def(gdef_3, input_map={"sub:0": mul_2}, return_elements=["output:0"])
    
with combine_graph_2.as_default():
    with tf.Session() as sess:
        opts = tf.get_default_graph().get_operations()
        print("----- Replaced Graph -----")
        for opt in opts:
            for output in opt.outputs: print(output.name)
                
        print(sess.run([mul, mul_2, output], feed_dict={x:1}))

----- Replaced Graph -----
x:0
import/input:0
import/Const:0
import/Add:0
import/Const_1:0
import/mul:0
import/Const_2:0
import/sub:0
import/Const_3:0
import/Add_1:0
import/output:0
Const:0
Mul:0
import_1/input:0
import_1/Const:0
import_1/Add:0
import_1/Const_1:0
import_1/mul:0
import_1/Const_2:0
import_1/sub:0
import_1/Const_3:0
import_1/Add_1:0
import_1/output:0
[20.0, 40.0, 140.0]
